In [1]:
import time
import math
import collections
import pandas as pd
import numpy as np
import scipy.sparse
from sklearn import preprocessing

In [2]:
SUBMISSION = False
SUBMISSION_FILENAME = 'submission.csv'
TEST_FILENAME = 'test.csv'

In [3]:
target_playlists = pd.read_csv('../datasets/target_playlists.csv', sep='\t')
target_tracks = pd.read_csv('../datasets/target_tracks.csv', sep='\t')
tracks_final = pd.read_csv('../datasets/tracks_final.csv', sep='\t')
playlists_final = pd.read_csv('../datasets/playlists_final.csv', sep='\t')
train_final = pd.read_csv('../datasets/train_final.csv', sep='\t')

print('Successfully read data')

Successfully read data


In [4]:
def check_matrix(X, format='csc', dtype=np.float32):
    if format == 'csc' and not isinstance(X, scipy.sparse.csc_matrix):
        return X.tocsc().astype(dtype)
    elif format == 'csr' and not isinstance(X, scipy.sparse.csr_matrix):
        return X.tocsr().astype(dtype)
    elif format == 'coo' and not isinstance(X, scipy.sparse.coo_matrix):
        return X.tocoo().astype(dtype)
    elif format == 'dok' and not isinstance(X, scipy.sparse.dok_matrix):
        return X.todok().astype(dtype)
    elif format == 'bsr' and not isinstance(X, scipy.sparse.bsr_matrix):
        return X.tobsr().astype(dtype)
    elif format == 'dia' and not isinstance(X, scipy.sparse.dia_matrix):
        return X.todia().astype(dtype)
    elif format == 'lil' and not isinstance(X, scipy.sparse.lil_matrix):
        return X.tolil().astype(dtype)
    else:
        return X.astype(dtype)

In [5]:
def is_relevant(recommendation_item, validation_set):
    validation_item = validation_set.loc[validation_set['playlist_id'] == recommendation_item['playlist_id']]
    recommendation_item['recommendation'] = pd.Series(recommendation_item['recommendation'])\
                                                .isin(list(validation_item['track_id']))
    return recommendation_item


def precision(recommended_items_relevance):
    precision_scores = recommended_items_relevance.sum(axis=1) / recommended_items_relevance.shape[1]
    return precision_scores.mean()


def mAP(recommended_items_relevance):
    p_at_k = recommended_items_relevance.cumsum(axis=1) / (1 + np.arange(recommended_items_relevance.shape[1]))
    recommended_items_mAP = p_at_k.sum(axis=1) / recommended_items_relevance.shape[1]
    return recommended_items_mAP.mean()


def evaluate_recommendations(recommended_items, validation_set):
    items_relevance = recommended_items.apply(lambda recommendation_item: is_relevant(recommendation_item, validation_set), axis=1)
    recommended_items_relevance = pd.DataFrame(list(items_relevance['recommendation']), index=items_relevance['recommendation'].index)
    precision_score = precision(recommended_items_relevance)
    mAP_score = mAP(recommended_items_relevance)
    return precision_score, mAP_score

def evaluate(recommended_items, validation_set):
    print('Evaluating...')
    begin = time.time()
    precision_score, mAP_score = evaluate_recommendations(recommended_items, validation_set)
    print('Precision: {0:.{digits}f}, mAP: {1:.{digits}f}, took {2:.{digits}f}s'
          .format(precision_score, mAP_score, time.time() - begin, digits=5))

In [6]:
def str_tags_to_list(str_tags):
    if str_tags == '[]':
        return []
    return list(map(int, str_tags.replace('[', '').replace(']', '').replace(' ', '').split(',')))


def str_album_to_int(album):
    if album == '[]' or album == '[None]':
        return -1
    return int(album.replace('[', '').replace(']', ''))

In [7]:
def get_weight(i):
    if i == 0:
        return 2.5
    if i == 1:
        return 2.0
    if i == 2:
        return 1.6
    if i == 3:
        return 1.2
    if i == 4:
        return 1.0

def get_tracks_tags(track_final):
    track_tags_list = str_tags_to_list(track_final['tags'])
    return [[track_final['track_id'], track_tag, get_weight(i)] for i, track_tag in enumerate(track_tags_list)]

try:
    tracks_tags = pd.DataFrame.from_csv('../datasets/tags.csv')
except:
    tracks_tags = pd.concat([pd.DataFrame(data=get_tracks_tags(track_final), columns=['track_id', 'tag', 'weight']) for index, track_final in tracks_final.iterrows()])
    tracks_tags.to_csv('../datasets/tags.csv')

print('tracks_tags {}'.format(tracks_tags.shape))
print(tracks_tags.head(6))

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:18: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


tracks_tags (483501, 3)
   track_id     tag  weight
0   2972914   54087     2.5
1   2972914    1757     2.0
2   2972914    1718     1.6
3   2972914  116712     1.2
4   2972914  189631     1.0
0   2750239  189631     2.5


In [8]:
def get_track_album(track_final):
    track_album = str_album_to_int(track_final['album'])
    return [[track_final['track_id'], track_album]]

try:
    tracks_albums = pd.DataFrame.from_csv('../datasets/albums.csv')
except:
    tracks_albums = pd.concat([pd.DataFrame(data=get_track_album(track_final), columns=['track_id', 'album']) for index, track_final in tracks_final.iterrows()])
    # Remove tracks without album
    tracks_albums = tracks_albums[tracks_albums.album != -1]
    tracks_albums.to_csv('../datasets/albums.csv')

print('tracks_albums {}'.format(tracks_albums.shape))
print(tracks_albums.head(6))

tracks_albums (73244, 2)
   track_id  album
0   2972914      7
0   2750239      8
0   1550729      9
0   2169950      9
0   2256817      9
0   2561768     26


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [9]:
tracks_artist = pd.DataFrame()
tracks_artist['track_id'] = tracks_final['track_id']
tracks_artist['artist_id'] = tracks_final['artist_id']
print('tracks_artist {}'.format(tracks_artist.shape))
print(tracks_artist.head(6))

tracks_artist (100000, 2)
   track_id  artist_id
0   2972914        144
1   2750239        246
2   1550729        144
3   2169950        144
4   1903709        144
5   2256817        144


In [10]:
def get_playlist_titles(playlist_final):
    playlist_tags_list = str_tags_to_list(playlist_final['title'])
    return [[playlist_final['playlist_id'], playlist_tag] for playlist_tag in playlist_tags_list]

try:
    playlist_titles = pd.DataFrame.from_csv('../datasets/titles.csv')
except:
    playlist_titles = pd.concat([pd.DataFrame(data=get_playlist_titles(playlist_final), columns=['playlist_id', 'title']) for index, playlist_final in playlists_final.iterrows()])
    playlist_titles.to_csv('../datasets/titles.csv')

print('playlist_titles {}'.format(playlist_titles.shape))
print(playlist_titles.head(6))

playlist_titles (108382, 2)
   playlist_id  title
0       644838  12727
0      3120683    183
0      4278112  12389
1      4278112  18698
2      4278112  18925
3      4278112  11695


//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls


In [11]:
playlist_owner = pd.DataFrame()
playlist_owner['playlist_id'] = playlists_final['playlist_id']
playlist_owner['owner'] = playlists_final['owner']
print('playlist_owner {}'.format(playlist_owner.shape))
print(playlist_owner.head(6))

playlist_owner (57561, 2)
   playlist_id  owner
0       644838  41504
1      7577564  41504
2      3120683  44542
3      4278112  44542
4      8656823  44542
5     10961458  44542


In [12]:
print('Data info:')
print('Tracks with tags count: {}'.format(tracks_tags['track_id'].nunique()))
print('Unique tags count: {}\n'.format(tracks_tags['tag'].nunique()))

print('Tracks with album count: {}'.format(tracks_albums['track_id'].nunique()))
print('Unique album count: {}\n'.format(tracks_albums['album'].nunique()))

print('Tracks with artists count: {}'.format(tracks_artist['track_id'].nunique()))
print('Unique artists count: {}\n'.format(tracks_artist['artist_id'].nunique()))

print('Playlists with title count: {}'.format(playlist_titles['playlist_id'].nunique()))
print('Unique titles count: {}\n'.format(playlist_titles['title'].nunique()))

print('Playlist with owner count: {}'.format(playlist_owner['playlist_id'].nunique()))
print('Unique owner count: {}\n'.format(playlist_owner['owner'].nunique()))

print('Unique tracks count: {}'.format(tracks_final['track_id'].nunique()))
print('Unique playlist count: {}'.format(playlists_final['playlist_id'].nunique()))
print('Target tracks count: {}'.format(target_tracks['track_id'].nunique()))
print('Target playlists count: {}'.format(target_playlists['playlist_id'].nunique()))

Data info:
Tracks with tags count: 97211
Unique tags count: 31900

Tracks with album count: 73244
Unique album count: 27604

Tracks with artists count: 100000
Unique artists count: 17536

Playlists with title count: 52384
Unique titles count: 21064

Playlist with owner count: 57561
Unique owner count: 15264

Unique tracks count: 100000
Unique playlist count: 57561
Target tracks count: 32195
Target playlists count: 10000


In [13]:
track_id_le = preprocessing.LabelEncoder()
track_id_le.fit(list(tracks_final['track_id']))
print('track_id_le classes: {}'.format(len(track_id_le.classes_)))

playlist_id_le = preprocessing.LabelEncoder()
playlist_id_le.fit(list(playlists_final['playlist_id']))
print('playlist_id_le classes: {}'.format(len(playlist_id_le.classes_)))

title_le = preprocessing.LabelEncoder()
title_le.fit(list(playlist_titles['title']))
print('title_le classes: {}'.format(len(title_le.classes_)))

owner_le = preprocessing.LabelEncoder()
owner_le.fit(list(playlist_owner['owner']))
print('owner_le classes: {}'.format(len(owner_le.classes_)))

tags_le = preprocessing.LabelEncoder()
tags_le.fit(list(tracks_tags['tag']))
print('tags_le classes: {}'.format(len(tags_le.classes_)))

album_le = preprocessing.LabelEncoder()
album_le.fit(list(tracks_albums['album']))
print('album_le classes: {}'.format(len(album_le.classes_)))

artist_le = preprocessing.LabelEncoder()
artist_le.fit(list(tracks_artist['artist_id']))
print('artist_le classes: {}'.format(len(artist_le.classes_)))

track_id_le classes: 100000
playlist_id_le classes: 57561
title_le classes: 21064
owner_le classes: 15264
tags_le classes: 31900
album_le classes: 27604
artist_le classes: 17536


In [14]:
# Playlist and tracks that belong to them
target_playlists_and_tracks = pd.merge(target_playlists, train_final, on='playlist_id')
print('target_playlists_and_tracks {}'.format(target_playlists_and_tracks.shape))
print(target_playlists_and_tracks.head(10))

target_playlists_and_tracks (362661, 2)
   playlist_id  track_id
0     10024884   2879006
1     10024884   1532328
2     10024884   3027673
3     10024884   3236144
4     10024884   1563134
5     10024884    435345
6     10024884    353291
7     10024884    247331
8     10024884    161455
9     10024884   3338954


In [15]:
def split_training_data(train_final, target_playlists_and_tracks, random_state):
    validation_set = target_playlists_and_tracks.groupby(['playlist_id'])\
                        .apply(lambda x: x.sample(n=3, random_state=random_state))\
                        .reset_index(drop=True)
    df_concat = pd.concat([train_final, validation_set])
    training_set = df_concat.drop_duplicates(keep=False)
    return training_set, validation_set

# Split dataset - from all target playlists remove randomly 3 tracks
training_set, validation_set = split_training_data(train_final, target_playlists_and_tracks, random_state=5)
test_target_tracks = validation_set['track_id'].drop_duplicates(keep='first').to_frame()
test_target_tracks['transformed_track_id'] = track_id_le.transform(list(test_target_tracks['track_id']))
target_tracks['transformed_track_id'] = track_id_le.transform(list(target_tracks['track_id']))

training_set['transformed_track_id'] = track_id_le.transform(list(training_set['track_id']))
training_set['transformed_playlist_id'] = playlist_id_le.transform(list(training_set['playlist_id']))
target_playlists['transformed_playlist_id'] = playlist_id_le.transform(list(target_playlists['playlist_id']))

train_final['transformed_track_id'] = track_id_le.transform(list(train_final['track_id']))
train_final['transformed_playlist_id'] = playlist_id_le.transform(list(train_final['playlist_id']))

print('train_final: {}'.format(train_final.shape))
print(train_final.head(5))
print('training_set: {}'.format(training_set.shape))
print(training_set.head(5))
print('validation_set: {}'.format(validation_set.shape))
print(validation_set.head(5))
print('test_target_tracks: {}'.format(test_target_tracks.shape))
print(test_target_tracks.head(5))

train_final: (1040522, 4)
   playlist_id  track_id  transformed_track_id  transformed_playlist_id
0      3271849   2801526                 71058                     8890
1      5616275    727878                 19741                    22426
2     11267488   2805283                 71298                    54743
3     10103900   1515105                 36851                    46970
4      3836898   2945623                 76310                    11967
training_set: (1010522, 4)
   playlist_id  track_id  transformed_track_id  transformed_playlist_id
0      3271849   2801526                 71058                     8890
1      5616275    727878                 19741                    22426
2     11267488   2805283                 71298                    54743
3     10103900   1515105                 36851                    46970
4      3836898   2945623                 76310                    11967
validation_set: (30000, 2)
   playlist_id  track_id
0         7614   1609224
1     

In [16]:
unique_tracks_count = tracks_final['track_id'].nunique()

playlist_titles['transformed_playlist_id'] = playlist_id_le.transform(list(playlist_titles['playlist_id']))
playlist_titles['transformed_title_id'] = list(map(lambda x: x + unique_tracks_count,\
                                                   title_le.transform(list(playlist_titles['title']))))
print('playlist_titles')
print(playlist_titles.head(4))

playlist_owner['transformed_playlist_id'] = playlist_id_le.transform(list(playlist_owner['playlist_id']))
playlist_owner['transformed_owner'] = list(map(lambda x: x + unique_tracks_count + len(title_le.classes_),\
                                                   owner_le.transform(list(playlist_owner['owner']))))
print('playlist_owner')
print(playlist_owner.head(4))

tracks_tags['transformed_track_id'] = track_id_le.transform(list(tracks_tags['track_id']))
tracks_tags['transformed_tag'] = tags_le.transform(list(tracks_tags['tag']))
print('tracks_tags')
print(tracks_tags.head(3))

tracks_albums['transformed_track_id'] = track_id_le.transform(list(tracks_albums['track_id']))
tracks_albums['transformed_album'] = list(map(lambda x: x + len(tags_le.classes_),\
                                  album_le.transform(list(tracks_albums['album']))))
print('tracks_albums')
print(tracks_albums.head(3))

tracks_artist['transformed_track_id'] = track_id_le.transform(list(tracks_artist['track_id']))
tracks_artist['transformed_artist_id'] = list(map(lambda x: x + len(tags_le.classes_) + len(album_le.classes_),\
                                                  artist_le.transform(list(tracks_artist['artist_id']))))
print('tracks_artist')
print(tracks_artist.head(3))

playlist_titles
   playlist_id  title  transformed_playlist_id  transformed_title_id
0       644838  12727                     2514                112727
0      3120683    183                     7829                100183
0      4278112  12389                    14708                112389
1      4278112  18698                    14708                118698
playlist_owner
   playlist_id  owner  transformed_playlist_id  transformed_owner
0       644838  41504                     2514             132679
1      7577564  41504                    35844             132679
2      3120683  44542                     7829             135702
3      4278112  44542                    14708             135702
tracks_tags
   track_id    tag  weight  transformed_track_id  transformed_tag
0   2972914  54087     2.5                 77187             6683
1   2972914   1757     2.0                 77187              257
2   2972914   1718     1.6                 77187              254
tracks_albums
   t

In [17]:
if SUBMISSION:
    print(training_set.shape)
    training_set = train_final
    print(training_set.shape)

In [20]:
urm_with_title_ones = np.ones(training_set.shape[0] + playlist_titles.shape[0])
urm_with_all_ones = np.ones(training_set.shape[0] + playlist_titles.shape[0] + playlist_owner.shape[0])

print('urm_with_title_ones shape: {}, vector: {}'.format(urm_with_title_ones.shape, urm_with_title_ones))
print('urm_with_all_ones shape: {}, vector: {}'.format(urm_with_all_ones.shape, urm_with_all_ones))

URM_with_title = scipy.sparse.coo_matrix((urm_with_title_ones, (list(training_set['transformed_playlist_id']) + list(playlist_titles['transformed_playlist_id']),
                                                                list(training_set['transformed_track_id']) + list(playlist_titles['transformed_title_id']))))

URM_with_all = scipy.sparse.coo_matrix((urm_with_all_ones, (list(training_set['transformed_playlist_id']) + list(playlist_titles['transformed_playlist_id']) + list(playlist_owner['transformed_playlist_id']),
                                                            list(training_set['transformed_track_id']) + list(playlist_titles['transformed_title_id']) + list(playlist_owner['transformed_owner']))))

URM_with_title = URM_with_title.tocsr()
URM_with_all = URM_with_all.tocsr()

print(URM_with_all.shape)
tracks_per_playlist = (URM_with_all > 0).sum(axis=1)
playlist_per_track = (URM_with_all > 0).sum(axis=0)

print('tracks_per_playlist.shape {}'.format(URM_with_all.shape))
print('playlist_per_track.shape {}'.format(URM_with_all.shape))

urm_with_title_ones shape: (1118904,), vector: [ 1.  1.  1. ...,  1.  1.  1.]
urm_with_all_ones shape: (1176465,), vector: [ 1.  1.  1. ...,  1.  1.  1.]
(57561, 136328)
tracks_per_playlist.shape (57561, 136328)
playlist_per_track.shape (57561, 136328)


In [21]:
urm_ones = np.ones(training_set.shape[0])
print('urm_ones shape: {}, vector: {}'.format(urm_ones.shape, urm_ones))

URM = scipy.sparse.coo_matrix((urm_ones, (list(training_set['transformed_track_id']),
                                          list(training_set['transformed_playlist_id']))))
print(URM.shape)

urm_ones shape: (1010522,), vector: [ 1.  1.  1. ...,  1.  1.  1.]
(100000, 57561)


In [22]:
print('tracks_tags.shape {}'.format(tracks_tags.shape))
print('tracks_albums.shape {}'.format(tracks_albums.shape))
print('tracks_artist.shape {}'.format(tracks_artist.shape))

tracks_albums_ones = np.ones(tracks_albums.shape[0])
tracks_artist_ones = np.ones(tracks_artist.shape[0])
values = np.append(tracks_tags['weight'], np.append(tracks_albums_ones, tracks_artist_ones))
print('values shape: {}, vector: {}'.format(values.shape, values))

ICM = scipy.sparse.coo_matrix((values, (list(tracks_tags['transformed_track_id']) + list(tracks_albums['transformed_track_id']) + list(tracks_artist['transformed_track_id']),\
                                        list(tracks_tags['transformed_tag']) + list(tracks_albums['transformed_album']) + list(tracks_artist['transformed_artist_id']))))

print(ICM.shape)
features_per_item = (ICM > 0).sum(axis=1)
items_per_feature = (ICM > 0).sum(axis=0)

print('features_per_item.shape {}'.format(features_per_item.shape))
print('items_per_feature.shape {}'.format(items_per_feature.shape))

tracks_tags.shape (483501, 5)
tracks_albums.shape (73244, 4)
tracks_artist.shape (100000, 4)
values shape: (656745,), vector: [ 2.5  2.   1.6 ...,  1.   1.   1. ]
(100000, 77040)
features_per_item.shape (100000, 1)
items_per_feature.shape (1, 77040)


In [23]:
class ISimilarity(object):
    """Abstract interface for the similarity metrics"""

    def __init__(self, shrinkage=10):
        self.shrinkage = shrinkage

    def compute(self, X):
        pass


class Cosine(ISimilarity):
    def compute(self, X):
        # convert to csc matrix for faster column-wise operations
        X = check_matrix(X, 'csc', dtype=np.float32)

        # 1) normalize the columns in X
        # compute the column-wise norm
        # NOTE: this is slightly inefficient. We must copy X to compute the column norms.
        # A faster solution is to  normalize the matrix inplace with a Cython function.
        Xsq = X.copy()
        Xsq.data **= 2
        norm = np.sqrt(Xsq.sum(axis=0))
        norm = np.asarray(norm).ravel()
        norm += 1e-6
        # compute the number of non-zeros in each column
        # NOTE: this works only if X is instance of sparse.csc_matrix
        
        col_nnz = np.diff(X.indptr)
        # then normalize the values in each column
        X.data /= np.repeat(norm, col_nnz)
        print("Normalized")

        # 2) compute the cosine similarity using the dot-product
        print("Computing distance")
        dist = X * X.T
        print("Computed")

        # and apply the shrinkage
        if self.shrinkage > 0:
            dist = self.apply_shrinkage(X, dist)
            print("Applied shrinkage")    
        
        return dist

    def apply_shrinkage(self, X, dist):
        # create an "indicator" version of X (i.e. replace values in X with ones)
        X_ind = X.copy()
        X_ind.data = np.ones_like(X_ind.data)
        # compute the co-rated counts
        co_counts = X_ind * X_ind.T
        # remove the diagonal
#         co_counts = co_counts - scipy.sparse.dia_matrix((co_counts.diagonal()[scipy.newaxis, :], [0]), shape=co_counts.shape)
        # compute the shrinkage factor as co_counts_ij / (co_counts_ij + shrinkage)
        # then multiply dist with it
        co_counts_shrink = co_counts.copy()
        co_counts_shrink.data += self.shrinkage
        co_counts.data /= co_counts_shrink.data
        dist.data *= co_counts.data
        return dist

In [24]:
distance = Cosine(shrinkage=8)
playlist_similarity_title = distance.compute(URM_with_title.copy())

Normalized
Computing distance
Computed
Applied shrinkage


In [25]:
distance = Cosine(shrinkage=8)
playlist_similarity_all = distance.compute(URM_with_all.copy())

Normalized
Computing distance
Computed
Applied shrinkage


In [26]:
distance = Cosine(shrinkage=0)
items_similarity_ICM = distance.compute(ICM.copy())

Normalized
Computing distance
Computed


In [27]:
distance = Cosine(shrinkage=0.1)
items_similarity_URM = distance.compute(URM.copy())

Normalized
Computing distance
Computed
Applied shrinkage


In [28]:
item_scores = pd.Series.from_csv('./hybrid_weights/item_scores.csv')
content_scores = pd.Series.from_csv('./hybrid_weights/content_scores.csv')

user_with_all_scores = pd.Series.from_csv('./hybrid_weights/user_with_all.csv')
user_with_title_scores = pd.Series.from_csv('./hybrid_weights/user_with_title.csv')

mean_i_mAP = item_scores.mean()
mean_c_mAP = content_scores.mean()

mean_uto_mAP = user_with_all_scores.mean()
mean_ut_mAP = user_with_title_scores.mean()

print('{} {} {} {}'.format(mean_i_mAP, mean_c_mAP, mean_ut_mAP, mean_uto_mAP))

//anaconda/lib/python3.5/site-packages/pandas/core/series.py:2849: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  infer_datetime_format=infer_datetime_format)


0.026331008021518693 0.04821255959483566 0.031002219568091925 0.032309362264514654


In [29]:
target_playlists_and_tracks = pd.merge(target_playlists, training_set, on='playlist_id')
target_playlists_and_tracks = pd.merge(target_playlists_and_tracks, tracks_final, on='track_id')
print('target_playlists_and_tracks {}'.format(target_playlists_and_tracks.shape))
print(target_playlists_and_tracks.head(5))

target_playlists_and_tracks (332661, 10)
   playlist_id  transformed_playlist_id_x  track_id  transformed_track_id  \
0     10024884                      46297   2879006                 74168   
1     10024884                      46297   1532328                 37429   
2      8466002                      41478   1532328                 37429   
3        68290                        550   1532328                 37429   
4      3520148                      10188   1532328                 37429   

   transformed_playlist_id_y  artist_id  duration  playcount     album  \
0                      46297     337976    185000       14.0  [155542]   
1                      46297      43620    271000       37.0   [21332]   
2                      41478      43620    271000       37.0   [21332]   
3                        550      43620    271000       37.0   [21332]   
4                      10188      43620    271000       37.0   [21332]   

                                     tags  
0   [35

In [30]:
target_playlists['item_mAP'] = item_scores
target_playlists['content_mAP'] = content_scores
target_playlists['user_all_mAP'] = user_with_all_scores
target_playlists['user_title_mAP'] = user_with_title_scores

def sum_mAP(playlist):
    playlist['sum_mAP'] = playlist['item_mAP'] + playlist['user_title_mAP'] + playlist['content_mAP']
    return playlist

target_playlists = target_playlists.apply(lambda playlist: sum_mAP(playlist), axis=1)
target_playlists = target_playlists.sort_values(['sum_mAP'], ascending=False)
target_playlists_and_data = pd.merge(target_playlists, playlists_final, on='playlist_id')
target_playlists_and_data = target_playlists_and_data.drop('owner', 1)
target_playlists_and_data = target_playlists_and_data.drop('title', 1)

def known_numtracks(playlist):
    playlist['known_numtracks'] = len(target_playlists_and_tracks.loc[target_playlists_and_tracks['playlist_id'] == playlist['playlist_id']])
    return playlist

target_playlists_and_data = target_playlists_and_data.apply(lambda playlist: known_numtracks(playlist), axis=1)
print(target_playlists_and_data.head(3))

   playlist_id  transformed_playlist_id  item_mAP  content_mAP  user_all_mAP  \
0    6264193.0                  26637.0  0.308117     0.301665      0.303278   
1   10794082.0                  51986.0  0.298440     0.281773      0.284366   
2    5895355.0                  24318.0  0.259091     0.298440      0.283684   

   user_title_mAP   sum_mAP  created_at  numtracks  duration  known_numtracks  
0        0.301397  0.911179  1274730103         61     13311                4  
1        0.285190  0.865403  1219268098         14      4297                2  
2        0.287189  0.844719  1227053150         16      3249                2  


In [31]:
target_tracks_and_data = pd.merge(target_tracks, tracks_final, on='track_id')
target_tracks_and_data = target_tracks_and_data.sort_values(by=['duration'])
print(target_tracks_and_data.head(10))

       track_id  transformed_track_id  artist_id  duration  playcount album  \
25226   2209516                 55057     319839        -1      711.0    []   
1272    1187609                 28910      48894        -1      249.0    []   
1271    1507255                 36711     230462        -1      115.0    []   
5364     906185                 22939     168149        -1     4993.0    []   
11817    451847                 11899     268875        -1      632.0    []   
5375    2610563                 65506     168149        -1     1731.0    []   
1248    1758377                 43354     312689        -1        4.0    []   
1273     945403                 23711     406890        -1        0.0    []   
18765   3216726                 83496     206962        -1       21.0    []   
5417      23086                   456     392005        -1       14.0    []   

      tags  
25226   []  
1272    []  
1271    []  
5364    []  
11817   []  
5375    []  
1248    []  
1273    []  
18765   []  


In [32]:
def find_all_subsets(elems, length, expected_sum):
    response = []

    def find_all_subsets_rec(available_elems, acc_set):
        cur_sum = sum([x[0] for x in acc_set])

        if cur_sum > expected_sum:
            return
        if length == len(acc_set):
            if cur_sum == expected_sum:
                response.append(acc_set)
            return
        if len(available_elems) + len(acc_set) < length:
            return

        find_all_subsets_rec(available_elems[1:], acc_set + [available_elems[0]])
        find_all_subsets_rec(available_elems[1:], acc_set)

    find_all_subsets_rec(elems, [])
    response_with_scores = [(r, sum([x[2] for x in r])) for r in response]
    return list(sorted(response_with_scores, key=lambda x: x[1], reverse=True))

In [33]:
class HybridRecommender:
    def __init__(self, k, to_predict=3):
        self.k = k
        self.to_predict = to_predict
    
    def fit(self, training_set, target_tracks, playlist_similarity_all, playlist_similarity_title, URM, items_similarity_ICM, items_similarity_URM):
        self.training_set = training_set
        self.playlist_similarity_all = playlist_similarity_all
        self.playlist_similarity_title = playlist_similarity_title
        self.items_similarity_ICM = items_similarity_ICM
        self.items_similarity_URM = items_similarity_URM
        self.URM = check_matrix(URM, 'csc', dtype=np.float32)
        self.target_tracks_mask = np.zeros(self.items_similarity_ICM.shape[0])
        for value in list(target_tracks['transformed_track_id']):
            self.target_tracks_mask[value] = 1
    
    def recommend(self, target_playlists):
        def make_recommendation(playlist):
            tracks_on_playlist = self.training_set.loc[self.training_set['playlist_id'] == playlist['playlist_id']]
            transformed_tracks_on_playlist = track_id_le.transform(list(tracks_on_playlist['track_id']))
            tracks_on_playlist_mask = np.ones(self.items_similarity_ICM.shape[0])
            for value in transformed_tracks_on_playlist:
                tracks_on_playlist_mask[value] = 0
   
            # ContentBased similarity
            reg = 10e-2
            c_mAP = playlist['content_mAP'] + reg * mean_c_mAP
            ut_mAP = playlist['user_title_mAP'] + reg * mean_ut_mAP
            uto_mAP = playlist['user_all_mAP'] + reg * mean_uto_mAP
            i_mAP = playlist['item_mAP'] + reg * mean_i_mAP
          
            # Content similarity
            correlation_content = np.squeeze(np.asarray(self.items_similarity_ICM[:, transformed_tracks_on_playlist].mean(axis=1)))            
            correlation_content = correlation_content / max(correlation_content)

            # ItemBased similarity
            correlation_item = np.squeeze(np.asarray(self.items_similarity_URM[:, transformed_tracks_on_playlist].mean(axis=1)))
            correlation_item = correlation_item / max(correlation_item)
            
            # UserBased similarity
            if playlist['user_title_mAP'] > playlist['user_all_mAP'] * 1.1:
                uu_mAP = ut_mAP
                similar_playlists = np.squeeze(self.playlist_similarity_title.getcol(playlist['transformed_playlist_id']).toarray())
            else:
                uu_mAP = uto_mAP
                similar_playlists = np.squeeze(self.playlist_similarity_all.getcol(playlist['transformed_playlist_id']).toarray())

            indices = np.argpartition(similar_playlists, -self.k)[-self.k:]
            ub_scores = np.take(similar_playlists, indices)
            correlation_user = np.zeros(self.URM.shape[0])
            for index, score in zip(indices, ub_scores):
                correlation_user += np.squeeze(self.URM.getcol(index).toarray()) * score
            correlation_user = correlation_user / max(correlation_user)     
            
            correlation = correlation_content * c_mAP + correlation_item * i_mAP + correlation_user * uu_mAP
            correlation = correlation * self.target_tracks_mask
            correlation = correlation * tracks_on_playlist_mask
            
#             ind = np.argpartition(list(correlation), -self.to_predict)[-self.to_predict:]
            ind = np.argpartition(list(correlation), -7)[-7:]
            scores = np.take(correlation, ind)
            
            ind, scores = zip(*sorted(zip(ind, scores), key=lambda x: x[1], reverse=True))

            flag = True
            d = {'transformed_track_id': ind, 'score': scores}
            df = pd.DataFrame(data=d)
            df = pd.merge(df, target_tracks_and_data, on='transformed_track_id')
            tracks_on_playlist = target_playlists_and_tracks.loc[target_playlists_and_tracks['playlist_id'] == playlist['playlist_id']]
            
            if (playlist['numtracks'] - len(tracks_on_playlist) == 5):
                total_duration = playlist['duration']
                known_duration = tracks_on_playlist['duration'].sum() / 1000
                expected_duration = (total_duration - known_duration) * 1000
# # #                 expected_duration = validation_set_data.loc[validation_set_data['playlist_id'] == playlist['playlist_id']]['duration'].sum()
                
                isd = list(zip(df['duration'], df['track_id'], df['score']))
                subsets = find_all_subsets(elems=isd, length=5, expected_sum=expected_duration)

                if len(subsets) > 0:
                    best = subsets[0][0]
                    recommended_tracks = [best[0][1], best[1][1], best[2][1], best[3][1], best[4][1]]
                    scores = [best[0][2], best[1][2], best[2][2], best[3][2], best[4][2]]
                    flag = False

#             recommended_tracks = track_id_le.inverse_transform(ind)
            if flag:
                recommended_tracks = track_id_le.inverse_transform(ind[:5])
                scores = scores[:5]
            playlist['recommendation'] = list(recommended_tracks)
            playlist['scores'] = list(scores)
            return playlist

        recommended_items = target_playlists.apply(lambda playlist: make_recommendation(playlist), axis=1)
        return recommended_items

In [34]:
print('Building model...')
begin = time.time()
recommender = HybridRecommender(k=15, to_predict=3)
# recommender.fit(training_set, test_target_tracks, playlist_similarity_all, playlist_similarity_title, URM, items_similarity_ICM, items_similarity_URM)
recommender.fit(training_set, target_tracks, playlist_similarity_all, playlist_similarity_title, URM, items_similarity_ICM, items_similarity_URM)
print('Took {0:.{digits}f}s'.format(time.time() - begin, digits=5))

Building model...
Took 0.11622s


In [35]:
print('Recommending...')
begin = time.time()
recommended_items = recommender.recommend(target_playlists_and_data.head(100))

print('Took {0:.{digits}f}s'.format(time.time() - begin, digits=5))

print('recommended_items {}'.format(recommended_items.shape))
print(recommended_items.head(20))

Recommending...
Took 18.71687s
recommended_items (100, 13)
    playlist_id  transformed_playlist_id  item_mAP  content_mAP  user_all_mAP  \
0     6264193.0                  26637.0  0.308117     0.301665      0.303278   
1    10794082.0                  51986.0  0.298440     0.281773      0.284366   
2     5895355.0                  24318.0  0.259091     0.298440      0.283684   
3     5378504.0                  20851.0  0.282056     0.271343      0.283830   
4     6985944.0                  31100.0  0.252660     0.305429      0.272011   
5     7222096.0                  32566.0  0.256645     0.306504      0.267199   
6    10315407.0                  48780.0  0.279461     0.253440      0.290015   
7     3140454.0                   7980.0  0.265824     0.282364      0.231248   
8     7270644.0                  32821.0  0.258531     0.285905      0.268235   
9     8419128.0                  41154.0  0.252741     0.281235      0.253171   
10    6299114.0                  26865.0  0.257207

In [36]:
if not SUBMISSION:
    evaluate(recommended_items, validation_set)

Evaluating...
Precision: 0.20800, mAP: 0.23010, took 0.74911s


In [37]:
def print_results(recommended_items, filename):
    print('Printing...')
    with open('../submissions/{}'.format(filename), 'w') as output_file:
        output_file.write('playlist_id,track_ids\n')
        for index, recommendation in recommended_items.iterrows():
            row = '{},'.format(int(recommendation['playlist_id']))
            for track_id in pd.Series(recommendation['recommendation']).values:
                row += ' {}'.format(track_id)
            row += '\n'
            output_file.write(row)
print_results(recommended_items, filename=SUBMISSION_FILENAME if SUBMISSION else TEST_FILENAME)